<a href="https://colab.research.google.com/github/cyc1974/BPI2019/blob/main/%E3%80%8CKnowledgeDistillation_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 以CIFAR10 dataset十種類別分類作為範例

In [ ]:

import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D,GlobalAveragePooling2D,Dense,Softmax,Flatten,MaxPooling2D,Dropout,Activation, Lambda, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import kullback_leibler_divergence as KLD_Loss, categorical_crossentropy as logloss
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import categorical_accuracy
import seaborn as sns

NUM_CLASSES = 10
class_names = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]     
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
 
# Normalize the dataset
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train shape: (50000, 32, 32, 3) y_train shape: (50000, 1)


# Teacher model

In [ ]:
Teacher = Sequential() 
Teacher.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', input_shape=(32,32,3)))
Teacher.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
Teacher.add(MaxPooling2D(pool_size=2))
Teacher.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
Teacher.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
Teacher.add(MaxPooling2D(pool_size=2))
Teacher.add(Dropout(0.5))
Teacher.add(Flatten())
Teacher.add(Dense(512, activation='relu'))
Teacher.add(Dropout(0.5))
Teacher.add(Dense(10))
Teacher.add(Activation('softmax'))

Teacher.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
 
Teacher.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 16)        208       
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        2080      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 32)        4128      
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        8256      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                    

# Teacher model training

In [ ]:
myCP = ModelCheckpoint(save_best_only=True,filepath='teacher.h5',monitor = 'accuracy')
Teacher.fit(x_train,
         y_train,
         batch_size=128,
         epochs=100,
         validation_split = 0.2,
         callbacks=[myCP])

Epoch 1/100
313/313 [==============================] - 63s 197ms/step - loss: 1.6379 - accuracy: 0.4004 - val_loss: 1.3481 - val_accuracy: 0.5182
Epoch 2/100
313/313 [==============================] - 62s 197ms/step - loss: 1.2792 - accuracy: 0.5396 - val_loss: 1.1337 - val_accuracy: 0.5938
Epoch 3/100
313/313 [==============================] - 63s 201ms/step - loss: 1.1406 - accuracy: 0.5906 - val_loss: 1.0241 - val_accuracy: 0.6457
Epoch 4/100
313/313 [==============================] - 65s 209ms/step - loss: 1.0429 - accuracy: 0.6286 - val_loss: 0.9601 - val_accuracy: 0.6608
Epoch 5/100
313/313 [==============================] - 58s 186ms/step - loss: 0.9750 - accuracy: 0.6544 - val_loss: 0.8931 - val_accuracy: 0.6871
Epoch 6/100
299/313 [===========================>..] - ETA: 2s - loss: 0.9075 - accuracy: 0.6787